In [2]:
%store -r \
endpoint_name \
sagemaker_session_bucket

In [3]:
import sagemaker
sm_session = sagemaker.session.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
import os
import uuid
import json


def generate_json(data):
    suffix = str(uuid.uuid1())
    filename = f'payload_{suffix}.json'
    with open(filename, 'w') as fp:
        json.dump(data, fp)
    return filename


def upload_input(sm_session, filename):
    return sm_session.upload_data(
        filename,
        bucket=sm_session.default_bucket(),
        key_prefix='musicgen/input_payload',
        extra_args={"ContentType": "application/json"},
    )


def delete_file_on_disk(filename):
    if os.path.isfile(filename):
        os.remove(filename)

In [5]:
import urllib, time
from botocore.exceptions import ClientError
import random

def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    icons = ["🎵","🎶","🎷","🎸","🎺","🎼","🥁"]
    print("generating music")
    while True:
        try:
            res = sm_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
            print("\nMusic is ready!🎉")
            return res
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                
                print(random.choice(icons), end = '')
                time.sleep(2)
                continue
            raise
    

import botocore
def download_from_s3(url):
    """ex: url = s3://bucketname/prefix1/music.wav"""
    url_parts = url.split("/")  # => ['s3:', '', 'sagemakerbucketname', 'data', ...
    bucket_name = url_parts[2]
    key = os.path.join(*url_parts[3:])
    filename = url_parts[-1]
    if not os.path.exists(filename):
        try:
            # Create an S3 client
            s3 = boto3.resource('s3')
            print('Downloading {} to {}'.format(url, filename))
            s3.Bucket(bucket_name).download_file(key, filename)
            return filename
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print('The object {} does not exist in bucket {}'.format(
                    key, bucket_name))
            else:
                raise
                
                
import IPython
def play_output_audio(filename):
    return IPython.display.Audio(filename)


In [6]:
data = {
    "texts": ['Morning sunshine, bets, ukelele, happy swings'],
    "bucket_name": sagemaker_session_bucket
}

In [7]:
filename = generate_json(data)
input_s3_location = upload_input(sm_session, filename)
delete_file_on_disk(filename)

In [8]:
import boto3
sagemaker_runtime = boto3.client('sagemaker-runtime')
response = sagemaker_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name,
    InputLocation=input_s3_location,
    ContentType="application/json",
)

In [9]:
response

{'ResponseMetadata': {'RequestId': 'b2d557ea-9669-44f7-83df-1306e62633e5',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': 'b2d557ea-9669-44f7-83df-1306e62633e5',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-west-2-920487201358/musicgen/async_inference/music_output/f76f9842-aaaf-4c78-8b4d-f707dcca084a.out',
   'x-amzn-sagemaker-failurelocation': 's3://sagemaker-us-west-2-920487201358/async-endpoint-failures/musicgen-small-v1-asyc-2023-10-29-18-37-48-730-1698604670-8b0b/f76f9842-aaaf-4c78-8b4d-f707dcca084a-error.out',
   'date': 'Sun, 29 Oct 2023 19:09:27 GMT',
   'content-type': 'application/json',
   'content-length': '54',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-west-2-920487201358/musicgen/async_inference/music_output/f76f9842-aaaf-4c78-8b4d-f707dcca084a.out',
 'FailureLocation': 's3://sagemaker-us-west-2-920487201358/async-endpoint-failures/musicgen-small-v1-asyc-2023-10-29-18-37-48-730-1698604670-8b0b/

In [10]:
output = get_output(response.get('OutputLocation'))
output = json.loads(output)
output.keys()

generating music
🎷🎺🎷🥁🎺🎺🥁🎵🎺🎷🎺🎵🎸🎷🎵🎺🥁🎺🎼🥁🎺
Music is ready!🎉


dict_keys(['generated_output_s3', 'audio_values'])

In [11]:
output.get('generated_output_s3')

's3://sagemaker-us-west-2-920487201358/musicgen/output/musicgen_out-c6768264-768e-11ee-84a6-2a97cd5712b3.wav'

In [12]:
music = download_from_s3(output.get('generated_output_s3'))

In [13]:
play_output_audio(music)

In [15]:
delete_file_on_disk(music)

## Cleanup

In [ ]:
sm_client = boto3.client('sagemaker')
endpoint = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = endpoint['EndpointConfigName']
endpoint_config = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = endpoint_config['ProductionVariants'][0]['ModelName']

print(f"""
About to delete the following sagemaker resources:
Endpoint: {endpoint_name}
Endpoint Config: {endpoint_config_name}
Model: {model_name}
""")

In [ ]:
# delete endpoint
#sm_client.delete_endpoint(EndpointName=endpoint_name)
# delete endpoint config
#sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# delete model
#sm_client.delete_model(ModelName=model_name)